In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/06-best-practices/documents-with-ids.json #use this cell to download the document file from repo

In [4]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [6]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/home/codespace/.python/current/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/948 [00:00<?, ?it/s]

In [58]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [59]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Hybrid search example


In [11]:
course = "data-engineering-zoomcamp"


In [12]:
query = 'I just discovered the course. Can I still join it?'


In [14]:
v_q = model.encode(query)


In [20]:
knn_query = {
    "field": "text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
    "boost": 0.5,
    "filter": {
        "term": {
            "course": course
        }
    }
}

In [17]:
keyword_query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^3", "text", "section"],
                "type": "best_fields",
                "boost": 0.5,
            }
        },
        "filter": {
            "term": {
                "course": course
            }
        }
    }
}

In [26]:
response = es_client.search(
    index=index_name,
    query=keyword_query,
    knn=knn_query,
    size=5
)

In [36]:
response["hits"]['hits'][0]

{'_index': 'course-questions',
 '_id': 'WoVGdpEBJ-GlSoSIruxb',
 '_score': 36.424633,
 '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a',
  'question_vector': [0.0030358789954334497,
   -0.0023871944285929203,
   0.0358816422522068,
   0.020998863503336906,
   -0.018282365053892136,
   0.06715094298124313,
   -0.10277319699525833,
   -0.11509548872709274,
   -0.06606756150722504,
   -0.004973309114575386,
   -0.002861772431060672,
   0.10543150454759598,
   -0.0008143176091834903,
   0.08418367058038712,
   0.02704712562263012,
   -0.031353775411844254,
   -0.05154323950409889,
   -0.049489956349134445,
   0.0534985102713108

# Hybrid search pipeline


In [37]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/06-best-practices/ground-truth-data.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-08-21 18:56:23--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/06-best-practices/ground-truth-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 504747 (493K) [text/plain]
Saving to: ‘ground-truth-data.csv’

ground-truth-data.c 100%[===================>] 492.92K  2.55MB/s    in 0.2s    

2024-08-21 18:56:24 (2.55 MB/s) - ‘ground-truth-data.csv’ saved [504747/504747]



In [38]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')


In [39]:
ground_truth = df_ground_truth.to_dict(orient='records')


In [41]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [42]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [43]:
def elastic_search_hybrid(field, query, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [44]:
def question_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid('question_vector', question, v_q, course)

In [45]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [46]:
evaluate(ground_truth, question_hybrid)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9234925437648585, 'mrr': 0.8481665586052878}

In [47]:
def text_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid('text_vector', question, v_q, course)

In [48]:
evaluate(ground_truth, text_hybrid)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9234925437648585, 'mrr': 0.8461710251422809}

In [49]:
def question_text_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid('question_text_vector', question, v_q, course)



In [50]:
evaluate(ground_truth, question_text_hybrid)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9250054030689432, 'mrr': 0.8506231539514445}

# Reranking

In [51]:
def elastic_search_hybrid_rrf(field, query, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "rank": {
            "rrf": {}
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [52]:
course = "data-engineering-zoomcamp"


In [54]:
query = 'I just discovered the course. Can I still join it?'


In [53]:
v_q = model.encode(query)


In [ ]:
elastic_search_hybrid_rrf('question_text_vector', query, v_q, course) #needs paid option

# RRF implementation

In [56]:
def compute_rrf(rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(field, query, vector, course, k=60):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    knn_results = es_client.search(
        index=index_name, 
        body={
            "knn": knn_query, 
            "size": 10
        }
    )['hits']['hits']
    
    keyword_results = es_client.search(
        index=index_name, 
        body={
            "query": keyword_query, 
            "size": 10
        }
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:5]:
        doc = es_client.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])
    
    return final_results

In [60]:
def question_text_hybrid_rrf(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('question_text_vector', question, v_q, course)

evaluate(ground_truth, question_text_hybrid_rrf)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9520207477847418, 'mrr': 0.8745911677833017}